In [3]:
import pandas as pd
import numpy as np


In [4]:
#uvoz podataka
#sezona 23/24
df_23_4 = pd.read_csv("bundesliga-2023.csv")
#sezona 24/25
df_24_5 = pd.read_csv("bundesliga-2024.csv")
#sezona 25/6
df_25_6 = pd.read_csv("bundesliga-2025.csv")

In [5]:
#za sezonu 25 6
df_23_4["Home Team"] = df_23_4["Home Team"].replace("Sport-Club Freiburg", 'SC Freiburg')
df_23_4["Away Team"] = df_23_4["Away Team"].replace("Sport-Club Freiburg", 'SC Freiburg')
#za sezonu 24 5
df_25_6["Home Team"] = df_25_6["Home Team"].replace("Sport-Club Freiburg", 'SC Freiburg')
df_25_6["Away Team"] = df_25_6["Away Team"].replace("Sport-Club Freiburg", 'SC Freiburg')

## Obavezno!!!

In [6]:
#odrediti duzinu sezone
df_25_6 = df_25_6.head(108)
df_25_6.tail()

,Match Number,Round Number,Date,Location,Home Team,Away Team,Result
103,108,12,29/11/2025 14:30,Allianz Arena,FC Bayern München,FC St. Pauli,3 - 1
104,107,12,29/11/2025 17:30,BayArena,Bayer 04 Leverkusen,Borussia Dortmund,1 - 2
105,100,12,30/11/2025 14:30,Volksparkstadion,Hamburger SV,VfB Stuttgart,2 - 1
106,106,12,30/11/2025 16:30,Deutsche Bank Park,Eintracht Frankfurt,VfL Wolfsburg,1 - 1
107,105,12,30/11/2025 18:30,Europa-Park Stadion,SC Freiburg,1. FSV Mainz 05,4 - 0


## Priprema Podataka i df-a

In [7]:
#sjedinjavanje ta tri df niza podataka - po osi jedan - odnosno uzduž
df_BL = pd.concat([df_23_4, df_24_5, df_25_6], axis=0)
df_BL = df_BL.drop(["Match Number","Round Number","Date","Location"],axis=1)
df_BL.rename(columns={"Home Team":"H","Away Team":"A"},inplace=True)

rez_dom = df_BL["Result"].str.slice(0,1)
rez_gost = df_BL["Result"].str.slice(4,5)

df_rezultatski = pd.concat([df_BL,rez_dom,rez_gost],axis=1)
df_rezultatski.columns = ["H","A","Result","Rez1","Rez2"]

df1 = df_rezultatski

## Definisanje Funckije za Izracun Forme

In [8]:
def klub(tim):
    # Filtriramo DataFrame
    df = df1[(df1["H"] == tim) | (df1["A"] == tim)].copy()
    
    uslovi = [
    (df['H'] == tim) & (df['Rez1'] < df["Rez2"]),
    (df['A'] == tim) & (df['Rez1'] < df["Rez2"]),
    (df['H'] == tim) & (df['Rez1'] == df["Rez2"]),
    (df['A'] == tim) & (df['Rez1'] == df["Rez2"]),
    (df['H'] == tim) & (df['Rez1'] > df["Rez2"]),
    (df['A'] == tim) & (df['Rez1'] > df["Rez2"]),]
    
    vrijednosti = [-1, 1, 0, 0, 1, -1]
    df['Forma'] = np.select(uslovi, vrijednosti, default=0)
    df = df.drop(["Rez1","Rez2"],axis=1).tail(5)
    rezultat = df["Forma"].sum() / len(df["Forma"])
    return rezultat

## Provjera Rada te Funckije

In [9]:
klub('FC Augsburg')

-0.6

## Stvaranje Petlje za Automatski Izracun Forme Svih Klubova

In [10]:
#izvlacenje naziva svih klubova
timovi = pd.concat([df1["H"], df1["A"]]).unique()
timovi

array(['SV Werder Bremen', 'VfB Stuttgart', 'FC Augsburg',
       'TSG Hoffenheim', 'VfL Wolfsburg', 'Bayer 04 Leverkusen',
       'Borussia Dortmund', '1. FC Union Berlin', 'Eintracht Frankfurt',
       'RB Leipzig', 'SV Darmstadt 98', '1. FC Heidenheim 1846',
       'VfL Bochum 1848', '1. FC Köln', 'SC Freiburg',
       'Borussia Mönchengladbach', '1. FSV Mainz 05', 'FC Bayern München',
       'FC St. Pauli', 'Holstein Kiel', 'Hamburger SV'], dtype=object)

In [11]:
rezultati = {}

In [12]:
#petlja
for tim in timovi:
    rezultat = klub(tim)
    rezultati[tim] = rezultat

# Kreiranje DataFrame-a iz rezultata
df_forme = pd.DataFrame(list(rezultati.items()),columns = ["Klub","Rezultat"])
df_forme["Rezultat"] = df_forme["Rezultat"]

## Rezulat Forme za Sve

In [13]:
#Izracun
df_forme 

,Klub,Rezultat
0,SV Werder Bremen,0.2
1,VfB Stuttgart,0.0
2,FC Augsburg,-0.6
3,TSG Hoffenheim,0.8
4,VfL Wolfsburg,-0.4
5,Bayer 04 Leverkusen,0.2
6,Borussia Dortmund,0.6
7,1. FC Union Berlin,-0.2
8,Eintracht Frankfurt,0.6
9,RB Leipzig,0.4


In [14]:
#Ponovna provjera vrijednosti...
klub("FC St. Pauli")

-1.0

In [15]:
df_forme.to_csv("forma_h.csv")

In [16]:
df_forme["Rezultat"] = df_forme["Rezultat"] * -1

In [17]:
df_forme


,Klub,Rezultat
0,SV Werder Bremen,-0.2
1,VfB Stuttgart,-0.0
2,FC Augsburg,0.6
3,TSG Hoffenheim,-0.8
4,VfL Wolfsburg,0.4
5,Bayer 04 Leverkusen,-0.2
6,Borussia Dortmund,-0.6
7,1. FC Union Berlin,0.2
8,Eintracht Frankfurt,-0.6
9,RB Leipzig,-0.4


In [18]:
df_forme.to_csv("forma_a.csv")